In [ ]:
import pandas as pd 
import spacy
import os
from tqdm import tqdm 
import json
import pickle
from sentence_transformers import SentenceTransformer
import torch
import string

In [ ]:
modelPath = "../../Data/Bert_Embedder"

model = SentenceTransformer('oliverguhr/german-sentiment-bert')
model.save(modelPath)

In [3]:
path_to_data = "../../Data/Email_Data/sympany_emails_de.jsonl"
sympany_email_de = pd.read_json(path_or_buf=path_to_data, lines=True)

In [7]:
def clean_data(text:string):
    text = text.replace('\n', ' ')
    # remove these characters "! , ."
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [8]:
def sentence2words(text:string):
    cleaned_text = clean_data(text.lower())
    lst_words = cleaned_text.split()
    return lst_words

In [9]:
def process(corpus):
    word_corpus = []
    for index, row in tqdm(corpus.iterrows()):
        id, text = row
        lst_words = sentence2words(text)
        word_corpus.append(lst_words)
    return word_corpus   

In [ ]:
cleaned_data_path = "../../Data/Preprocessed_Data"
file_path = os.path.join(cleaned_data_path, "email2words.pickle")

email_to_lst_words = process(sympany_email_de)

with open(file_path, "wb") as file: 
    pickle.dump(email_to_lst_words, file)


In [ ]:
# create sentence (email) embeddings
# load the pretrained sentenceTransformer for mulit language
modelPath = "../../Data/Bert_Embedder"
embeddings_model = SentenceTransformer(modelPath)

path_to_data = "../../Data/Email_Data/sympany_emails_de.jsonl"
sympany_email_de = pd.read_json(path_or_buf=path_to_data, lines=True)

email_datset = sympany_email_de["text"]

sentence_embedding = []
for each_email in tqdm(email_datset):
    email_embedded = embeddings_model.encode(each_email.lower())
    sentence_embedding.append(email_embedded)
    
sentence_embedding = torch.tensor(sentence_embedding)
sentence_embedding.shape

# save as pickle file
cleaned_data_path = "../../Data/Preprocessed_Data"
file_path = os.path.join(cleaned_data_path, "email_embeddings.pickle")
with open(file_path, "wb") as file:
    pickle.dump(sentence_embedding, file)


